In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
home = pd.read_csv('home_completed.csv')
away = pd.read_csv('away_completed.csv')

In [3]:
#put all data into one large data frame
main = home.append(away, ignore_index=True).sort_values(['Checker']) # puts into date order
#put columns into more organized order
main = main[['Year','Month', 'Day' , 'Team', 'Spread', 'Line', '+/-', '3P', '3P%', '3PA', '3PAr', 'AST', 'BLK', 'DRB', 'Day',
       'FG', 'FG%', 'FGA', 'FT', 'FT%', 'FTA', 'FTr', 'Home', 'Losses',
        'ORB', 'PF', 'PTS', 'STL', 'Starters MP', 'TOV', 'TS%', 'Wins',
        'eFG%',  'Checker']]
#number of points a team won or lost by
main['+/-'] = main['+/-']/5
#puts the odds into floats
main['Spread'] = main['Spread'].replace(' PK', '0').astype(float)
#gets how many points a team went over or under the spread
main['Cover'] = main['Spread'] + main['+/-']
#true or false on whether a team covered the spread
main['ATS'] = np.where(main['Cover'].isnull(), np.nan,
          np.where(main['Cover'] > 0,   1, 0))
teams = list(main['Team'])
new_teams = []
#adds the opponent to the df
for i in range(0, len(teams), 2):
    new_teams.append(teams[i+1])
    new_teams.append(teams[i])
main['Opponent'] = new_teams
main = main.dropna()

In [4]:
def team_df(team):
    t = main[main['Team']== team].reset_index()
    del t['index']
    t['Season'] = t['Checker'].apply(season) #define what season a game is played
    t['Games Played'] = t['Wins'] + t['Losses'] #total games played
    #create running averages for all stats up to a given game date
    t_df = pd.DataFrame()
    for year in years:
        temp = t[t['Season']==year].reset_index()
        temp['Avg Win/Loss'] = (temp['+/-'].cumsum() - temp['+/-'])/temp.index
        temp['Avg 3P'] = (temp['3P'].cumsum() - temp['3P'])/temp.index
        temp['Avg 3PA'] = (temp['3PA'].cumsum() - temp['3PA'])/temp.index
        temp['Avg 3P%'] = (temp['Avg 3P']/temp['Avg 3PA'])
        temp['Avg Starter MP'] = (temp['Starters MP'].cumsum() - temp['Starters MP'])/temp.index
        temp['Avg AST'] = (temp['AST'].cumsum() - temp['AST'])/temp.index
        temp['Avg BLK'] = (temp['BLK'].cumsum() - temp['BLK'])/temp.index
        temp['Avg DRB'] = (temp['DRB'].cumsum() - temp['DRB'])/temp.index
        temp['Avg ORB'] = (temp['ORB'].cumsum() - temp['ORB'])/temp.index
        temp['Avg FG'] = (temp['FG'].cumsum() - temp['FG'])/temp.index
        temp['Avg FGA'] = (temp['FGA'].cumsum() - temp['FGA'])/temp.index
        temp['Avg FG%'] = (temp['Avg FG']/temp['Avg FGA'])
        temp['Avg FT'] = (temp['FT'].cumsum() - temp['FT'])/temp.index
        temp['Avg FTA'] = (temp['FTA'].cumsum() - temp['FTA'])/temp.index
        temp['Avg 3PAr'] = (temp['Avg 3PA']/temp['Avg FGA'])
        temp['Avg FTr'] = (temp['Avg FTA']/temp['Avg FGA'])
        temp['Avg 3P%'] = (temp['Avg FT']/temp['Avg FTA'])/temp.index
        temp['Avg PF'] = (temp['PF'].cumsum() - temp['PF'])/temp.index
        temp['Avg PTS'] = (temp['PTS'].cumsum() - temp['PTS'])/temp.index
        temp['Avg STL'] = (temp['STL'].cumsum() - temp['STL'])/temp.index
        temp['Avg TOV'] = (temp['TOV'].cumsum() - temp['TOV'])/temp.index
        temp['Avg TS%'] = temp['Avg PTS']/ (2*(temp['Avg FGA'] + .44 * temp['Avg FTA']))
        temp['Avg eFG%'] = (temp['Avg FG'] + .5 * temp['Avg 3P'])/ temp['Avg FGA']
        temp['Avg Starter MP'] = (temp['Starters MP'].cumsum() - temp['Starters MP'])/temp.index
        temp['Winning Percentage'] = temp['Wins']/temp.index
        t_df = t_df.append(temp)
    return t_df

In [5]:
def team_df(team):
    t = main[main['Team']== team].reset_index()
    del t['index']
    t['Season'] = t['Checker'].apply(season) #define what season a game is played
    t['Games Played'] = t['Wins'] + t['Losses'] #total games played
    #create running averages for all stats up to a given game date
    t_df = pd.DataFrame()
    for year in years:
        temp = t[t['Season']==year].reset_index()
        temp['Avg Win/Loss'] = (temp['+/-'].cumsum() - temp['+/-'])/temp.index
        temp['Avg 3P'] = (temp['3P'].cumsum() - temp['3P'])/temp.index
        temp['Avg 3PA'] = (temp['3PA'].cumsum() - temp['3PA'])/temp.index
        temp['Avg 3P%'] = (temp['Avg 3P']/temp['Avg 3PA'])
        temp['Avg Starter MP'] = (temp['Starters MP'].cumsum() - temp['Starters MP'])/temp.index
        temp['Avg AST'] = (temp['AST'].cumsum() - temp['AST'])/temp.index
        temp['Avg BLK'] = (temp['BLK'].cumsum() - temp['BLK'])/temp.index
        temp['Avg DRB'] = (temp['DRB'].cumsum() - temp['DRB'])/temp.index
        temp['Avg ORB'] = (temp['ORB'].cumsum() - temp['ORB'])/temp.index
        temp['Avg FG'] = (temp['FG'].cumsum() - temp['FG'])/temp.index
        temp['Avg FGA'] = (temp['FGA'].cumsum() - temp['FGA'])/temp.index
        temp['Avg FG%'] = (temp['Avg FG']/temp['Avg FGA'])
        temp['Avg FT'] = (temp['FT'].cumsum() - temp['FT'])/temp.index
        temp['Avg FTA'] = (temp['FTA'].cumsum() - temp['FTA'])/temp.index
        temp['Avg 3PAr'] = (temp['Avg 3PA']/temp['Avg FGA'])
        temp['Avg FTr'] = (temp['Avg FTA']/temp['Avg FGA'])
        temp['Avg 3P%'] = (temp['Avg FT']/temp['Avg FTA'])/temp.index
        temp['Avg PF'] = (temp['PF'].cumsum() - temp['PF'])/temp.index
        temp['Avg PTS'] = (temp['PTS'].cumsum() - temp['PTS'])/temp.index
        temp['Avg STL'] = (temp['STL'].cumsum() - temp['STL'])/temp.index
        temp['Avg TOV'] = (temp['TOV'].cumsum() - temp['TOV'])/temp.index
        temp['Avg TS%'] = temp['Avg PTS']/ (2*(temp['Avg FGA'] + .44 * temp['Avg FTA']))
        temp['Avg eFG%'] = (temp['Avg FG'] + .5 * temp['Avg 3P'])/ temp['Avg FGA']
        temp['Winning Percentage'] = temp['Wins']/temp.index
        ###
        temp['5 Game Win/Loss'] = temp['+/-'].rolling(5, min_periods=1).sum()-temp['+/-']
        temp['5 Game 3P'] = temp['3P'].rolling(5, min_periods=1).sum() - temp['3P']
        temp['5 Game 3PA'] = temp['3PA'].rolling(5, min_periods=1).sum()- temp['3PA']
        temp['5 Game 3P%'] = temp['5 Game 3P']/temp['5 Game 3PA']
        temp['5 Game Starter MP'] = temp['Starters MP'].rolling(5, min_periods=1).sum()-temp['Starters MP']
        temp['5 Game AST'] = temp['AST'].rolling(5, min_periods=1).sum() - temp['AST']
        temp['5 Game BLK'] = temp['BLK'].rolling(5, min_periods=1).sum() - temp['BLK']
        temp['5 Game DRB'] = temp['DRB'].rolling(5, min_periods=1).sum() - temp['DRB']
        temp['5 Game ORB'] = temp['ORB'].rolling(5, min_periods=1).sum() - temp['ORB']
        temp['5 Game FG'] = temp['FG'].rolling(5, min_periods=1).sum() - temp['FG']
        temp['5 Game FGA'] = temp['FGA'].rolling(5, min_periods=1).sum() - temp['FGA']
        temp['5 Game FG%'] = temp['5 Game FG']/temp['5 Game FGA']
        temp['5 Game FT'] = temp['FT'].rolling(5, min_periods=1).sum() - temp['FT']
        temp['5 Game FTA'] = temp['FTA'].rolling(5, min_periods=1).sum() - temp['FTA']
        temp['5 Game 3PAr'] = temp['5 Game 3PA']/temp['5 Game FGA']
        temp['5 Game FTr'] = temp['5 Game FTA']/temp['5 Game FGA']
        temp['5 Game 3P%'] = temp['5 Game FT']/temp['5 Game FTA']
        temp['5 Game PF'] = temp['PF'].rolling(5, min_periods=1).sum() - temp['PF']
        temp['5 Game PTS'] = temp['PTS'].rolling(5, min_periods=1).sum() - temp['PTS']
        temp['5 Game STL'] = temp['STL'].rolling(5, min_periods=1).sum() - temp['STL']
        temp['5 Game TOV'] = temp['TOV'].rolling(5, min_periods=1).sum() - temp['TOV']
        temp['5 Game TS%'] = temp['5 Game PTS']/ (2*(temp['5 Game FGA'] + .44 * temp['5 Game FTA']))
        temp['5 Game eFG%'] = (temp['5 Game FG'] + .5 * temp['5 Game 3P'])/ temp['5 Game FGA']
        t_df = t_df.append(temp)
    return t_df

In [6]:
#put each row into a season
def season(date):
    date = date[:6]
    if date < '200809':
        return '2007'
    elif date < '200909':
        return '2008'
    elif date < '201009':
        return '2009'
    elif date < '201109':
        return '2010'
    elif date < '201109':
        return '2010'
    elif date < '201209':
        return '2011'
    elif date < '201309':
        return '2012'
    elif date < '201409':
        return '2013'
    elif date < '201509':
        return '2014'
    elif date < '201609':
        return '2015'
    elif date < '201709':
        return '2016'
    elif date < '201809':
        return '2017'
    elif date < '201909':
        return '2018'
    

In [7]:
teams = set(teams)
years = ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
all_team_data = {}
for team in teams:
    all_team_data[team] = team_df(team)

In [8]:
giant_df = pd.DataFrame()
for df in all_team_data.values():
    giant_df = giant_df.append(df)

In [9]:
for team in teams:
    t = all_team_data[team]
    opp = giant_df[giant_df['Opponent']==team]
    opp = opp.rename(columns = {'Avg Win/Loss':'oppAvg Win/Loss', 'Avg 3P':'oppAvg 3P', 'Avg 3PA':'oppAvg 3PA' ,
       'Avg 3P%':'oppAvg 3P%', 'Avg Starter MP':'oppAvg Starter MP', 'Avg AST':'oppAvg AST', 'Avg BLK':'oppAvg BLK', 'Avg DRB':'oppAvg DRB', 'Avg ORB':'oppAvg ORB',
       'Avg FG':'oppAvg FG', 'Avg FGA':'oppAvg FGA', 'Avg FG%':'oppAvg FG%', 'Avg FT':'oppAvg FT', 'Avg FTA':'oppAvg FTA', 'Avg 3PAr':'oppAvg 3PAr',
       'Avg FTr':'oppAvg FTr', 'Avg PF':'oppAvg PF', 'Avg PTS':'oppAvg PTS', 'Avg STL':'oppAvg STL', 'Avg TOV':'oppAvg TOV', 'Avg TS%':'oppAvg TS%',
       'Avg eFG%':'oppAvg eFG%', 'Winning Percentage':'oppWinning Percentage'})
    opp = opp.rename(columns = {'5 Game Win/Loss':'opp 5 Game Win/Loss', '5 Game 3P':'opp 5 Game 3P', '5 Game 3PA':'opp 5 Game 3PA',
       '5 Game 3P%':'opp 5 Game 3P%', '5 Game Starter MP':'opp 5 Game Starter MP', '5 Game AST':'opp 5 Game AST', '5 Game BLK':'opp 5 Game BLK',
       '5 Game DRB':'opp 5 Game DRB', '5 Game ORB':'opp 5 Game ORB',
       '5 Game FG':'opp 5 Game FG', '5 Game FGA':'opp 5 Game FGA', '5 Game FG%':'opp 5 Game FG%', '5 Game FT':'opp 5 Game FT', '5 Game FTA':'opp 5 Game FTA', '5 Game 3PAr':'opp 5 Game 3PAr',
       '5 Game FTr':'opp 5 Game FTr', '5 Game PF':'opp 5 Game PF', '5 Game PTS':'opp 5 Game PTS', '5 Game STL':'opp 5 Game STL', '5 Game TOV':'opp 5 Game TOV', '5 Game TS%':'opp 5 Game TS%',
       '5 Game eFG%':'opp 5 Game eFG%'})
    opp = opp[['Checker', 'oppAvg Win/Loss', 'oppAvg 3P', 'oppAvg 3PA',
       'oppAvg 3P%', 'oppAvg Starter MP', 'oppAvg AST', 'oppAvg BLK',
       'oppAvg DRB', 'oppAvg ORB', 'oppAvg FG', 'oppAvg FGA', 'oppAvg FG%',
       'oppAvg FT', 'oppAvg FTA', 'oppAvg 3PAr', 'oppAvg FTr', 'oppAvg PF',
       'oppAvg PTS', 'oppAvg STL', 'oppAvg TOV', 'oppAvg TS%', 'oppAvg eFG%',
       'oppWinning Percentage', 'opp 5 Game Win/Loss', 'opp 5 Game 3P', 'opp 5 Game 3PA',
       'opp 5 Game 3P%', 'opp 5 Game Starter MP', 'opp 5 Game AST', 'opp 5 Game BLK',
       'opp 5 Game DRB', 'opp 5 Game ORB', 'opp 5 Game FG', 'opp 5 Game FGA', 'opp 5 Game FG%',
       'opp 5 Game FT', 'opp 5 Game FTA', 'opp 5 Game 3PAr', 'opp 5 Game FTr', 'opp 5 Game PF',
       'opp 5 Game PTS', 'opp 5 Game STL', 'opp 5 Game TOV', 'opp 5 Game TS%', 'opp 5 Game eFG%']]
    all_team_data[team] = pd.merge(t,opp)

In [10]:
phi = all_team_data['Philadelphia']

In [11]:
total_matrix = phi[['Cover', 'ATS', '+/-', 'Wins', 'Losses', 'Home', 'Avg Win/Loss', 'Avg 3P',
       'Avg 3PA', 'Avg 3P%', 'Avg Starter MP', 'Avg AST', 'Avg BLK', 'Avg DRB',
       'Avg ORB', 'Avg FG', 'Avg FGA', 'Avg FG%', 'Avg FT', 'Avg FTA',
       'Avg 3PAr', 'Avg FTr', 'Avg PF', 'Avg PTS', 'Avg STL', 'Avg TOV',
       'Avg TS%', 'Avg eFG%', 'Winning Percentage', 'oppAvg Win/Loss','oppAvg 3P',
        'oppAvg 3PA', 'oppAvg 3P%', 'oppAvg Starter MP',
        'oppAvg AST', 'oppAvg BLK', 'oppAvg DRB', 'oppAvg ORB', 'oppAvg FG',
        'oppAvg FGA', 'oppAvg FG%', 'oppAvg FT', 'oppAvg FTA', 'oppAvg 3PAr',
        'oppAvg FTr', 'oppAvg PF', 'oppAvg PTS', 'oppAvg STL', 'oppAvg TOV',
        'oppAvg TS%', 'oppAvg eFG%', 'oppWinning Percentage', '5 Game Win/Loss', '5 Game 3P', '5 Game 3PA',
       '5 Game 3P%', '5 Game Starter MP', '5 Game AST', '5 Game BLK',
       '5 Game DRB', '5 Game ORB', '5 Game FG', '5 Game FGA', '5 Game FG%',
       '5 Game FT', '5 Game FTA', '5 Game 3PAr', '5 Game FTr', '5 Game PF',
       '5 Game PTS', '5 Game STL', '5 Game TOV', '5 Game TS%', '5 Game eFG%',
       'opp 5 Game Win/Loss', 'opp 5 Game 3P', 'opp 5 Game 3PA',
       'opp 5 Game 3P%', 'opp 5 Game Starter MP', 'opp 5 Game AST', 'opp 5 Game BLK',
       'opp 5 Game DRB', 'opp 5 Game ORB', 'opp 5 Game FG', 'opp 5 Game FGA', 'opp 5 Game FG%',
       'opp 5 Game FT', 'opp 5 Game FTA', 'opp 5 Game 3PAr', 'opp 5 Game FTr', 'opp 5 Game PF',
       'opp 5 Game PTS', 'opp 5 Game STL', 'opp 5 Game TOV', 'opp 5 Game TS%', 'opp 5 Game eFG%']]
total_matrix = total_matrix.dropna()


In [12]:
feature_matrix = total_matrix[['Wins', 'Losses', 'Home', 'Avg Win/Loss', 'Avg 3P',
       'Avg 3PA', 'Avg 3P%', 'Avg Starter MP', 'Avg AST', 'Avg BLK', 'Avg DRB',
       'Avg ORB', 'Avg FG', 'Avg FGA', 'Avg FG%', 'Avg FT', 'Avg FTA',
       'Avg 3PAr', 'Avg FTr', 'Avg PF', 'Avg PTS', 'Avg STL', 'Avg TOV',
       'Avg TS%', 'Avg eFG%', 'Winning Percentage']]
results = total_matrix['Cover']


In [13]:
feature_matrix = total_matrix[['Avg 3P%', 'Avg Starter MP', 'Avg AST', 'Avg ORB', 'Avg FG%', 'Avg FT', 'Avg FTA',
                               'Avg 3PAr', 'Avg FTr', 'Avg PF', 'Avg TOV',
                               'Avg TS%', 'Avg eFG%', 'Winning Percentage', 'oppAvg Win/Loss','oppAvg 3P',
                               'oppAvg 3PA', 'oppAvg 3P%', 'oppAvg Starter MP',
                               'oppAvg AST', 'oppAvg BLK', 'oppAvg DRB', 'oppAvg ORB', 'oppAvg FG',
                               'oppAvg FGA', 'oppAvg FG%', 'oppAvg FT', 'oppAvg FTA', 'oppAvg 3PAr',
                               'oppAvg FTr', 'oppAvg PF', 'oppAvg PTS', 'oppAvg STL', 'oppAvg TOV',
                               'oppAvg TS%', 'oppAvg eFG%', 'oppWinning Percentage']]

In [14]:
feature_matrix = total_matrix[['Avg ORB', 'Avg FTA',
                               'Avg 3PAr', 'Avg TOV',
                            'Avg eFG%', 'Winning Percentage', 
                               'oppAvg ORB', 'oppAvg FTA', 'oppAvg 3PAr',
                               'oppAvg TOV', 'oppAvg eFG%', 'oppWinning Percentage']]

In [15]:
feature_matrix = total_matrix[['Wins', 'Losses', 'Home', 'Avg Win/Loss', 'Avg 3P',
       'Avg 3PA', 'Avg 3P%', 'Avg Starter MP', 'Avg AST', 'Avg BLK', 'Avg DRB',
       'Avg ORB', 'Avg FG', 'Avg FGA', 'Avg FG%', 'Avg FT', 'Avg FTA',
       'Avg 3PAr', 'Avg FTr', 'Avg PF', 'Avg PTS', 'Avg STL', 'Avg TOV',
       'Avg TS%', 'Avg eFG%', 'Winning Percentage', 'oppAvg Win/Loss','oppAvg 3P',
        'oppAvg 3PA', 'oppAvg 3P%', 'oppAvg Starter MP',
        'oppAvg AST', 'oppAvg BLK', 'oppAvg DRB', 'oppAvg ORB', 'oppAvg FG',
        'oppAvg FGA', 'oppAvg FG%', 'oppAvg FT', 'oppAvg FTA', 'oppAvg 3PAr',
        'oppAvg FTr', 'oppAvg PF', 'oppAvg PTS', 'oppAvg STL', 'oppAvg TOV',
        'oppAvg TS%', 'oppAvg eFG%', 'oppWinning Percentage', '5 Game Win/Loss', '5 Game 3P', '5 Game 3PA',
       '5 Game 3P%', '5 Game Starter MP', '5 Game AST', '5 Game BLK',
       '5 Game DRB', '5 Game ORB', '5 Game FG', '5 Game FGA', '5 Game FG%',
       '5 Game FT', '5 Game FTA', '5 Game 3PAr', '5 Game FTr', '5 Game PF',
       '5 Game PTS', '5 Game STL', '5 Game TOV', '5 Game TS%', '5 Game eFG%',
       'opp 5 Game Win/Loss', 'opp 5 Game 3P', 'opp 5 Game 3PA',
       'opp 5 Game 3P%', 'opp 5 Game Starter MP', 'opp 5 Game AST', 'opp 5 Game BLK',
       'opp 5 Game DRB', 'opp 5 Game ORB', 'opp 5 Game FG', 'opp 5 Game FGA', 'opp 5 Game FG%',
       'opp 5 Game FT', 'opp 5 Game FTA', 'opp 5 Game 3PAr', 'opp 5 Game FTr', 'opp 5 Game PF',
       'opp 5 Game PTS', 'opp 5 Game STL', 'opp 5 Game TOV', 'opp 5 Game TS%', 'opp 5 Game eFG%']]

In [16]:
feature_matrix = total_matrix[['Wins', 'Losses', 'Home', 'Avg Win/Loss', 'Avg 3P',
       'Avg 3PA', 'Avg Starter MP', 'Avg AST', 'Avg BLK', 'Avg DRB',
       'Avg ORB', 'Avg FG', 'Avg FGA',
       'Avg 3PAr', 'Avg FTr', 'Avg PF', 'Avg PTS', 'Avg STL', 'Avg TOV',
       'Avg eFG%', 'oppAvg Win/Loss','oppAvg 3P',
        'oppAvg 3PA', 'oppAvg Starter MP',
        'oppAvg AST', 'oppAvg BLK', 'oppAvg DRB', 'oppAvg ORB', 'oppAvg FG',
        'oppAvg FGA', 'oppAvg 3PAr',
        'oppAvg FTr', 'oppAvg PF', 'oppAvg PTS', 'oppAvg STL', 'oppAvg TOV',
        'oppAvg eFG%', 'oppWinning Percentage', '5 Game Win/Loss', '5 Game 3P', '5 Game 3PA',
       '5 Game Starter MP', '5 Game AST', '5 Game BLK',
       '5 Game DRB', '5 Game ORB', '5 Game FG', '5 Game FGA', 
       '5 Game 3PAr', '5 Game FTr', '5 Game PF',
       '5 Game PTS', '5 Game STL', '5 Game TOV', '5 Game eFG%',
       'opp 5 Game Win/Loss', 'opp 5 Game 3P', 'opp 5 Game 3PA',
       'opp 5 Game Starter MP', 'opp 5 Game AST', 'opp 5 Game BLK',
       'opp 5 Game DRB', 'opp 5 Game ORB', 'opp 5 Game FG', 'opp 5 Game FGA', 
       'opp 5 Game 3PAr', 'opp 5 Game FTr', 'opp 5 Game PF',
       'opp 5 Game PTS', 'opp 5 Game STL', 'opp 5 Game TOV', 'opp 5 Game eFG%']]

In [17]:
from sklearn import linear_model
from sklearn.metrics import make_scorer
model = linear_model.LinearRegression()
model.fit(feature_matrix.iloc[:700], results.iloc[:700])
predictions = model.predict(feature_matrix[700:])

In [18]:
def accuracy_predictor(lst1, lst2):
    lst = []
    for i in range(len(lst1)):
        x = (lst1[i]<0)
        y = (lst2[i]<0)
        if(lst1[i]==0 or lst2[i]==0):
            pass
        elif x == y:
            lst.append(1)
        else:
            lst.append(0)
    return lst

In [19]:
accuracy = accuracy_predictor(predictions, results.tolist()[700:])

In [20]:
sum(accuracy)/len(accuracy)

0.4634146341463415

In [21]:
import xgboost as xgb
from sklearn.grid_search import GridSearchCV

/Users/ericweltz/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/ericweltz/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [22]:
results = total_matrix['ATS']

In [23]:
xgbmat = xgb.DMatrix(feature_matrix.iloc[:700], results.iloc[:700])

In [24]:
cv_params = {'max_depth': [5,7, 9], 'min_child_weight': [5, 7, 9], 
             'learning_rate': [0.1, .01, 0.005, .001], 'n_estimators' : [50, 100, 500]}
ind_params = {'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 'objective':'reg:linear'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1)

In [25]:
optimized_GBM.fit(feature_matrix.iloc[:700], results.iloc[:700])

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [5, 7, 9], 'min_child_weight': [5, 7, 9], 'learning_rate': [0.1, 0.01, 0.005, 0.001], 'n_estimators': [50, 100, 500]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [26]:
optimized_GBM.grid_scores_

[mean: 0.46286, std: 0.06260, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 50},
 mean: 0.45571, std: 0.04507, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 100},
 mean: 0.46000, std: 0.04989, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 500},
 mean: 0.48000, std: 0.06744, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 7, 'n_estimators': 50},
 mean: 0.45571, std: 0.05940, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 7, 'n_estimators': 100},
 mean: 0.45000, std: 0.04576, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 7, 'n_estimators': 500},
 mean: 0.43429, std: 0.04922, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 9, 'n_estimators': 50},
 mean: 0.44429, std: 0.05166, params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 9, 'n_estimators': 100},
 mean: 0.44429, std: 0.0540

In [27]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':7, 'min_child_weight':7, 'n_estimators': 500} 
# Grid Search CV optimized settings

xgdmat = xgb.DMatrix(feature_matrix.iloc[:700], results.iloc[:700])

cv_xgb = xgb.cv(params = our_params, dtrain = xgdmat, num_boost_round = 3000, nfold = 10,
                metrics = ['error'], # Make sure you enter metrics inside a list or you may encounter issues!
                early_stopping_rounds = 100) # Look for early stopping

In [28]:
cv_xgb

,test-error-mean,test-error-std,train-error-mean,train-error-std
0,0.498572,0.040633,0.269207,0.022851
1,0.492857,0.046181,0.205714,0.016666
2,0.491428,0.057214,0.163651,0.022068
3,0.477143,0.062007,0.134127,0.023653
4,0.497143,0.045535,0.113175,0.019762
5,0.492857,0.042499,0.091905,0.014932
6,0.502857,0.024578,0.082540,0.014754
7,0.502857,0.032451,0.073016,0.011358
8,0.495714,0.037280,0.062698,0.012483
9,0.490000,0.037280,0.053809,0.012608


In [67]:
final_gb = xgb.train(our_params, xgdmat, num_boost_round = 77)

In [68]:
y = final_gb.predict(xgb.DMatrix(feature_matrix.iloc[700:]))
y = [round(i) for i in y]
z = results.iloc[700:]

In [69]:
def accuracy_predictor2(lst1, lst2):
    lst = []
    for i in range(len(lst1)):
        x = (lst1[i]<0)
        y = (lst2[i]<0)
        if(lst1[i]==lst2[i]):
            lst.append(1)
        else:
            lst.append(0)
    return lst

In [70]:
accuracy2 = accuracy_predictor2(y, z.tolist())
print(sum(accuracy2)/len(accuracy2))

0.496


In [33]:
y[70]

0.0

In [34]:
z[770]

1.0

In [35]:
yy = pd.DataFrame(y, index=z.index)

In [36]:
yy = yy.sort_values(by=0)

In [37]:
colspace = yy.index[-20:]
colspace

Int64Index([802, 767, 766, 908, 807, 761, 759, 757, 756, 755, 919, 752, 751,
            749, 746, 929, 743, 742, 819, 712],
           dtype='int64')

In [41]:
dfsd = pd.DataFrame([1,2,3,4,5])

In [50]:
coeff = pd.DataFrame(model.coef_, feature_matrix.columns)
coeff.sort_values(by=0)

,0
Avg eFG%,-1277.978958
oppAvg 3PAr,-428.635537
opp 5 Game eFG%,-89.128027
Avg FGA,-9.422609
oppAvg FG,-4.376269
opp 5 Game FTr,-4.212049
Avg 3PAr,-3.878754
Avg BLK,-2.932029
oppAvg 3P,-1.922377
Avg TOV,-1.781388


In [43]:
feature_matrix.columns

Index(['Wins', 'Losses', 'Home', 'Avg Win/Loss', 'Avg 3P', 'Avg 3PA',
       'Avg Starter MP', 'Avg AST', 'Avg BLK', 'Avg DRB', 'Avg ORB', 'Avg FG',
       'Avg FGA', 'Avg 3PAr', 'Avg FTr', 'Avg PF', 'Avg PTS', 'Avg STL',
       'Avg TOV', 'Avg eFG%', 'oppAvg Win/Loss', 'oppAvg 3P', 'oppAvg 3PA',
       'oppAvg Starter MP', 'oppAvg AST', 'oppAvg BLK', 'oppAvg DRB',
       'oppAvg ORB', 'oppAvg FG', 'oppAvg FGA', 'oppAvg 3PAr', 'oppAvg FTr',
       'oppAvg PF', 'oppAvg PTS', 'oppAvg STL', 'oppAvg TOV', 'oppAvg eFG%',
       'oppWinning Percentage', '5 Game Win/Loss', '5 Game 3P', '5 Game 3PA',
       '5 Game Starter MP', '5 Game AST', '5 Game BLK', '5 Game DRB',
       '5 Game ORB', '5 Game FG', '5 Game FGA', '5 Game 3PAr', '5 Game FTr',
       '5 Game PF', '5 Game PTS', '5 Game STL', '5 Game TOV', '5 Game eFG%',
       'opp 5 Game Win/Loss', 'opp 5 Game 3P', 'opp 5 Game 3PA',
       'opp 5 Game Starter MP', 'opp 5 Game AST', 'opp 5 Game BLK',
       'opp 5 Game DRB', 'opp 5 Game ORB', 